# **600k Fitness, Exercise & Workout Programs:**

In [1]:
!pip install kagglehub


In [2]:
import kagglehub

# Download dataset
path = kagglehub.dataset_download("adnanelouardi/600k-fitness-exercise-and-workout-program-dataset")

print("Path to dataset files:", path)


100%|██████████| 6.27M/6.27M [00:00<00:00, 65.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/adnanelouardi/600k-fitness-exercise-and-workout-program-dataset/versions/1


In [3]:
import os

os.listdir(path)


['program_summary.csv', 'programs_detailed_boostcamp_kaggle.csv']

Load program_summary.csv

In [4]:
import pandas as pd

summary_path = path + "/program_summary.csv"

df = pd.read_csv(summary_path)

df.head()


,title,description,level,goal,equipment,program_length,time_per_workout,total_exercises,created,last_edit
0,(MASS MONSTER) High Intensity 4 Day Upper Lowe...,Build tones of muscular with this high intensi...,['Intermediate'],"['Muscle & Sculpting', 'Bodyweight Fitness']",Full Gym,12.0,90.0,384,2024-01-20 10:23:00,2025-06-29 12:39:00
1,(NOT MY PROGRAM)SHJ Jotaro,Build strength and size,"['Advanced', 'Intermediate']",['Bodybuilding'],Full Gym,8.0,60.0,224,2024-07-08 02:28:00,2025-06-18 09:15:00
2,1 PowerLift Per Day Powerbuilding 5 Day Bro Split,Based off of Andy Baker's KCS (Kingwood Streng...,"['Beginner', 'Novice', 'Intermediate']","['Athletics', 'Powerlifting', 'Powerbuilding']",Full Gym,6.0,90.0,237,2025-04-23 09:21:00,2025-06-18 11:55:00
3,10 Week Mass Building Program,This workout is designed to increase your musc...,"['Intermediate', 'Advanced']",['Powerbuilding'],Garage Gym,10.0,70.0,280,2024-09-07 03:44:00,2025-06-18 08:01:00
4,10 week deadlift focus,Increase deadlift,"['Intermediate', 'Advanced']","['Powerbuilding', 'Powerlifting', 'Bodybuildin...",Full Gym,10.0,80.0,356,2024-12-23 03:13:00,2025-06-18 12:19:00


**Check what columns exist**

In [5]:
df.columns


Index(['title', 'description', 'level', 'goal', 'equipment', 'program_length',
       'time_per_workout', 'total_exercises', 'created', 'last_edit'],
      dtype='object')

In [6]:
df.shape


(2598, 10)

**Clean the text**

In [7]:
df["description"].isna().sum()


np.int64(4)

In [8]:
df["description"].head(10)


,description
0,Build tones of muscular with this high intensi...
1,Build strength and size
2,Based off of Andy Baker's KCS (Kingwood Streng...
3,This workout is designed to increase your musc...
4,Increase deadlift
5,3 day a week push up program gradually increas...
6,To hit 1000lbs!
7,100x push up’s per day for 12 weeks
8,Develop hypertrophy with minimal equipment wit...
9,Get Big and Strong


Clean the dataset

In [9]:
df = df.dropna(subset=["description"]).reset_index(drop=True)
df.shape


(2594, 10)

Create NLP input text

In [10]:
texts = df["description"].tolist()

len(texts)


2594

Install Transformers

In [11]:
!pip install transformers torch datasets accelerate


Load DistilBERT Sentiment Pipeline

In [12]:
from transformers import pipeline

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Test on 3 workout descriptions

In [13]:
sample_texts = texts[:3]

for t in sample_texts:
    print(t)
    print(sentiment_pipeline(t))
    print("-"*50)


Build tones of muscular with this high intensity 4 day upper lower workout routine.

Weekly Setup:
Day 1 - Upper Push Focus
Day 2 - Lower Quad Focus
Day 3 - Rest/Active Recovery
Day 4 - Upper Pull Focus
Day 5 - Rest/Active Recovery
Day 6 - Lower Hamstring Focus
Day 7 - Rest/Active Recovery 

Muscle Groups |Sets Per Week
Back - 10
Shoulders - 10
Chest - 9
Quads - 9
Hamstrings - 9
Glutes - 9
Biceps - 7
Triceps - 7
Lower Back - 4
Calves - 4
Abs - 4
[{'label': 'POSITIVE', 'score': 0.9894155263900757}]
--------------------------------------------------
Build strength and size
[{'label': 'POSITIVE', 'score': 0.9998553991317749}]
--------------------------------------------------
Based off of Andy Baker's KCS (Kingwood Strength & Conditioning) Powerbuilding program.

It focuses on increasing strength on one lift per day, including squat, deadlift, bench press, overhead press, and pull-up. Rest days are after squat and deadlift days.

1 - Shoulders & Triceps
2 - Back
3 - Quads & Calves
Rest
4 

Load a Summarization Model (T5)

In [14]:
summarizer = pipeline(
    "summarization",
    model="t5-small",
    tokenizer="t5-small"
)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


Test summarization on the same text

In [15]:
long_text = texts[0]

summary = summarizer(long_text, max_length=60, min_length=20, do_sample=False)

print("ORIGINAL:")
print(long_text)
print("\nSUMMARY:")
print(summary[0]["summary_text"])


Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


ORIGINAL:
Build tones of muscular with this high intensity 4 day upper lower workout routine.

Weekly Setup:
Day 1 - Upper Push Focus
Day 2 - Lower Quad Focus
Day 3 - Rest/Active Recovery
Day 4 - Upper Pull Focus
Day 5 - Rest/Active Recovery
Day 6 - Lower Hamstring Focus
Day 7 - Rest/Active Recovery 

Muscle Groups |Sets Per Week
Back - 10
Shoulders - 10
Chest - 9
Quads - 9
Hamstrings - 9
Glutes - 9
Biceps - 7
Triceps - 7
Lower Back - 4
Calves - 4
Abs - 4

SUMMARY:
weekly Setup: Day 1 - Upper Push Focus Day 2 - Lower Quad Focus Day 3 - Rest/Active recovery Day 4 . upper pull focus day 5 - rest/active recovery day 6 .


# **Stage-2: Standalone Agentic AI**

Define the Agent Brain

In [16]:
class SimpleAgent:
    def __init__(self):
        self.memory = {}

    def perceive(self, user_goal):
        print("Perceiving goal...")
        return user_goal.lower()

    def plan(self, goal):
        print("Planning tasks...")
        tasks = []
        if "sentiment" in goal:
            tasks.append("analyze_sentiment")
        if "summarize" in goal or "summary" in goal:
            tasks.append("summarize_text")
        if "recommend" in goal:
            tasks.append("generate_recommendation")
        return tasks

    def act(self, task, data):
        print(f"Executing task: {task}")
        # For now, we simulate results
        if task == "analyze_sentiment":
            return "POSITIVE"
        elif task == "summarize_text":
            return "This is a workout program summary."
        elif task == "generate_recommendation":
            return "This program is recommended for strength building."
        else:
            return "Unknown task"

    def remember(self, task, result):
        self.memory[task] = result

    def run(self, user_goal, data):
        goal = self.perceive(user_goal)
        tasks = self.plan(goal)

        for task in tasks:
            result = self.act(task, data)
            self.remember(task, result)

        print("\nAgent Memory:")
        print(self.memory)
        return self.memory


Test the Agent

In [17]:
agent = SimpleAgent()

user_goal = "Analyze the sentiment and summarize this workout"
agent.run(user_goal, "Some workout text")


Perceiving goal...
Planning tasks...
Executing task: analyze_sentiment
Executing task: summarize_text

Agent Memory:
{'analyze_sentiment': 'POSITIVE', 'summarize_text': 'This is a workout program summary.'}


{'analyze_sentiment': 'POSITIVE',
 'summarize_text': 'This is a workout program summary.'}

# **Hybrid Agentic NLP System**

Create a Hybrid Agent

In [18]:
class HybridAgent:
    def __init__(self, sentiment_model, summarizer):
        self.sentiment_model = sentiment_model
        self.summarizer = summarizer
        self.memory = {}

    def perceive(self, goal, text):
        return {"goal": goal.lower(), "text": text}

    def plan(self, perception):
        tasks = []
        if "sentiment" in perception["goal"]:
            tasks.append("sentiment")
        if "summarize" in perception["goal"] or "summary" in perception["goal"]:
            tasks.append("summary")
        return tasks

    def act(self, task, text):
        if task == "sentiment":
            result = self.sentiment_model(text)[0]
            return result
        elif task == "summary":
            result = self.summarizer(text, max_length=60, min_length=20, do_sample=False)[0]["summary_text"]
            return result

    def remember(self, task, result):
        self.memory[task] = result

    def run(self, goal, text):
        perception = self.perceive(goal, text)
        tasks = self.plan(perception)

        for task in tasks:
            output = self.act(task, perception["text"])
            self.remember(task, output)

        return self.memory


Run the Hybrid System

In [19]:
hybrid_agent = HybridAgent(sentiment_pipeline, summarizer)

goal = "Analyze the sentiment and summarize this workout"
text = texts[0]

output = hybrid_agent.run(goal, text)

output


Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'sentiment': {'label': 'POSITIVE', 'score': 0.9894155263900757},
 'summary': 'weekly Setup: Day 1 - Upper Push Focus Day 2 - Lower Quad Focus Day 3 - Rest/Active recovery Day 4 . upper pull focus day 5 - rest/active recovery day 6 .'}

What just happened (in one sentence)

An autonomous agent interpreted your goal and decided to run two transformer models in sequence to produce an intelligent response.

That is the definition of Agentic AI.

{
 *'sentiment': {'label': 'POSITIVE', 'score': *0.989},
 *'summary': 'weekly Setup: Day 1 - Upper Push *Focus ...'
}
This means:

The agent read the goal:

“Analyze the sentiment and summarize this workout”

It planned:

*  I need sentiment

*  I need summary

It executed:

*  Called DistilBERT → got POSITIVE

*  Called T5 → got summary

  It stored the results in memory

It returned a structured intelligent response

This is no longer just NLP — this is AI system design.

